In [44]:
BEST_OF = 2
SIDES = ["Both"]

In [45]:
import pandas as pd

PROCESSED_FILES_FOLDER = '../data/processed'
OUTPUT_PATH = f'{PROCESSED_FILES_FOLDER}/full_matches_train_data.csv'

## Definindo as colunas que serão usadas para treinar o modelo
team_map_results_columns_to_add  = ["PlusMinus", "Rating"]
team_map_results_columns_to_drop  = []

map_results = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in [BEST_OF]
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in team_map_results_columns_to_add
                for side in SIDES]

kast_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in [BEST_OF]
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Kast"]
                for side in SIDES]

adr_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in [BEST_OF]
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Adr"]
                for side in SIDES]

### Listando as colunas que serão usadas das partidas, como: Jogadores, Mortes, Assistências etc.
matches_train_data_columns = ['eventId', 'matchId', 'mapBestOf'] + map_results

print(f"Reading data from {PROCESSED_FILES_FOLDER}/matches_results.csv and {PROCESSED_FILES_FOLDER}/matches.csv")

matches_results_train_data = pd.read_csv('../data/raw/train/matches_results.csv')
matches_train_data = pd.read_csv('../data/raw/train/matches.csv')

matches_train_data = matches_train_data[matches_train_data_columns]

matches_results_train_data_columns = ['eventId', 'matchId', 'TeamOneScore', 'TeamTwoScore', 'teamOneWon', 'teamTwoWon']
matches_results_train_data = matches_results_train_data[[col for col in matches_results_train_data.columns if any(s in col for s in matches_results_train_data_columns)]]

## Combinando os dados gerais das partidas com os detalhes das partidas
full_matches_train_data = pd.merge(matches_results_train_data, matches_train_data, on='matchId', how='inner')

full_matches_train_data = full_matches_train_data[full_matches_train_data['mapBestOf'] == BEST_OF]

## Data Wrangling
full_matches_train_data.fillna(0, inplace=True)
full_matches_train_data.replace("Not Available", 0, inplace=True)
full_matches_train_data.drop(['eventId_y', 'eventId_x', 'matchId'], axis=1, inplace=True)

full_matches_train_data.drop(team_map_results_columns_to_drop, axis=1, inplace=True)
full_matches_train_data.drop(['mapBestOf'], axis=1, inplace=True)

## arredondando os valores de Kast e Adr
# for column in kast_columns:
#     full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x.strip('%')) / 100, 2) if str(x) != '0' else 0)
# for column in adr_columns:
#     full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x) / 100, 2))

## Convertendo as colunas para float
# for column in full_matches_train_data.columns:
#     full_matches_train_data[column] = full_matches_train_data[column].astype(float)

full_matches_train_data.to_csv(OUTPUT_PATH, index=False)

display(full_matches_train_data.head())
display(full_matches_train_data.shape)

print(f"Data saved to {OUTPUT_PATH}")

Reading data from ../data/processed/matches_results.csv and ../data/processed/matches.csv


/var/folders/10/t4cd3lc515j88jjkvk8hvrbw0000gn/T/ipykernel_54302/1907829675.py:37: DtypeWarning: Columns (35,36,37,41,42,43,44,45,46,47,48,49,50,51,52,56,57,58,59,60,61,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,83,84,85,89,90,91,92,93,94,95,96,97,98,99,100,104,105,106,107,108,109,113,114,115,116,117,118,119,120,121,122,123,124,128,129,130,131,132,133,137,138,139,140,141,142,143,144,145,146,147,148,152,153,154,155,156,157,161,162,163,164,165,166,167,168,169,170,171,172,176,177,178,179,180,181,185,186,187,188,189,190,191,192,193,194,195,196,200,201,202,203,204,205,209,210,211,212,213,214,215,216,217,218,219,220,224,225,226,227,228,229,233,234,235,236,237,238,239,240,241,242,243,244,248,249,250,251,252,253,257,258,259,260,261,262,263,264,265,266,267,268,272,273,274) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_train_data = pd.read_csv('../data/raw/train/matches.csv')


,teamOneWon,teamTwoWon,map2Team1SideBothPlayer1PlusMinus,map2Team1SideBothPlayer1Rating,map2Team1SideBothPlayer2PlusMinus,map2Team1SideBothPlayer2Rating,map2Team1SideBothPlayer3PlusMinus,map2Team1SideBothPlayer3Rating,map2Team1SideBothPlayer4PlusMinus,map2Team1SideBothPlayer4Rating,...,map2Team2SideBothPlayer1PlusMinus,map2Team2SideBothPlayer1Rating,map2Team2SideBothPlayer2PlusMinus,map2Team2SideBothPlayer2Rating,map2Team2SideBothPlayer3PlusMinus,map2Team2SideBothPlayer3Rating,map2Team2SideBothPlayer4PlusMinus,map2Team2SideBothPlayer4Rating,map2Team2SideBothPlayer5PlusMinus,map2Team2SideBothPlayer5Rating
0,True,False,8,1.62,6,1.6,6,1.47,2,1.44,...,-1,0.95,-7,0.66,-4,0.61,-4,0.56,-7,0.5
1,False,True,-7,0.73,-6,0.65,-9,0.54,-9,0.53,...,16,2.08,12,1.76,4,1.35,5,1.28,0,1.08
2,True,False,8,1.56,4,1.41,5,1.23,6,1.2,...,-4,1.0,-3,0.95,-5,0.9,-2,0.82,-11,0.4
4,False,True,-2,0.87,-3,0.83,-5,0.78,-9,0.65,...,10,1.7,5,1.33,4,1.19,6,1.19,0,1.07
5,True,False,9,1.46,5,1.32,1,1.12,0,0.93,...,3,1.2,-6,1.07,0,0.96,-5,0.77,-5,0.65


(2689, 22)

Data saved to ../data/processed/full_matches_train_data.csv
